In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
pth_project = Path('/home/jupyter/churn_models')
sys.path.insert(0, str(pth_project))
from core.utils.gcp import connect_bq_services, connect_storage_services

In [ ]:
# Set global vars
pth_project = Path('/home/jupyter/churn_models')
pth_data = pth_project / 'data' 
pth_query = pth_project / 'core' / 'pipelines' / 'queries' /'account_churn'
pth_local_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'  # this is gcp credential for common project if you are working on
pth_global_creds = pth_project / 'conf' / 'global' / 'project_config.yaml' # use this local credential for your own dev

In [ ]:
config = pth_project / 'core' / 'pipelines' / 'account_churn' / 'config.yaml'
params = safe_load(config.open())
gcp_config = safe_load(pth_global_creds.open())
bkt_id = params['bkt_id']

In [ ]:
proj_id = gcp_config['gcp-project-name']
dataset_id = gcp_config['dataset']
train_table_id = params['train_table_id']
score_table_id = params['score_table_id']
pred_out_table_id = params['pred_out_table_id']
print('we will use project_id:{} and dataset_id:{}'.format(proj_id,dataset_id))

In [ ]:
bq_client = connect_bq_services(gcp_config['gcp-project-name'])

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Prizm_UniqueLicense_DF=pd.read_csv('Prizm_Unique_License_201901_updated.csv')

In [ ]:
Prizm_UniqueLicense_DF.info()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'divg_churn_analysis_pr_dataset.Prizm_Unique_License_201901'

bq_table_instance= bq_client.load_table_from_dataframe(Prizm_UniqueLicense_DF, Table_BQ,job_config=config)

In [ ]:
Prizm_Geo_DF=pd.read_csv('PRIZM5_GeoProfile_updated.csv')

In [ ]:
Prizm_Geo_DF.info()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'divg_churn_analysis_pr_dataset.PRIZM5_E17BASHHD_GeoProfile_GEO'

bq_table_instance= bq_client.load_table_from_dataframe(Prizm_Geo_DF, Table_BQ,job_config=config)